<font color='blue'>Data Science aplicada a otimização do planejamento da manutenção</font>

## Importação de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import r2_score
import networkx as nx

## Importação de Dataframes

In [ ]:
df = pd.read_csv('base_vigente_2020.csv')

## Seleção de linhas relacionadas ao sistema de ar

In [ ]:
df_pos = df[df['class'] == 'pos']

## Conversão de elementos de string para float

In [ ]:
df_full = {}
df_num = {}
for key in df_pos.keys():
    if key == 'class':
        continue
    # Coluna 'aa_000' já está em formato float
    if key == 'aa_000':
        df_full[key] = df_pos[key]
        df_num[key] = np.array(df_pos[key])[np.where(np.invert(np.isnan(df_pos[key])))]
        continue
    
    lista_val = df_pos[key]
    
    full = np.array([float(val) if val.isnumeric() else np.nan for val in lista_val])
    
    num = full[np.where(np.invert(np.isnan(full)))]
    
    # Criação de dois dicionários: Um com valores nan e outro com apenas números
    # Filtra colunas com valores constantes
    if not len(set(num)) <= 1:
        df_full[key] = full
        df_num[key] = num

## Filtra Outliers

In [ ]:
df_outlier = {}
for key in df_full.keys():
    media = np.mean(df_num[key])
    
    # Retira valores que sejam 5 vezes maiores que a média de valores
    df_outlier[key] = np.array([val if val <= 5*media else np.nan for val in df_full[key]])

## Filtra possíveis valores que possam representar a coluna de custos

In [ ]:
# O valor deve estar entre $10 e $500 e a soma próximo de $37000 (Considerando $1 = R$5.16)
soma_real = []
for key in df_full.keys():
    soma = np.sum(df_num[key])
    media = np.mean(df_num[key])

    if (51.6 < media < 2580) and (soma < 200000):
        soma_real.append(key)

In [ ]:
# O valor deve estar entre $10 e $500 e a soma próximo de $37000
soma_dolar = []
for key in df_full.keys():
    soma = np.sum(df_num[key])
    media = np.mean(df_num[key])

    if (10 < media < 500) and (soma < 37000):
        soma_dolar.append(key)

## Análise de correlação entre colunas

In [ ]:
lista_r2 = []
lista_edge = []
G = nx.Graph()
for (i1, key1) in enumerate(df_outlier.keys()):
    for (i2, key2) in enumerate(df_outlier.keys()):
        if i1 >= i2:
            continue
        
        # Lista com valores nan de cada tabela
        nan_1 = np.isnan(df_outlier[key1])
        nan_2 = np.isnan(df_outlier[key2])

        # Lista com linhas que poderão ser utilizadas
        useful = np.invert(np.logical_or(nan_1, nan_2))

        lista_1 = df_outlier[key1][np.where(useful)]
        lista_2 = df_outlier[key2][np.where(useful)]

        # Filtra colunas com valores constantes
        if (len(set(lista_1)) <= 1) or (len(set(lista_2)) <= 1):
            continue

        # Filtra pares com menos que 15 amostras válidas
        if len(lista_1) <= 15:
            continue

        coef = np.polyfit(lista_1, lista_2, 1)

        predict = np.polyval(coef, lista_1)

        r2 = r2_score(lista_2, predict)

        # Relaciona colunas se R2 >= 0.8
        if r2 >= 0.8:
            lista_r2.append([key1, key2, r2])
            lista_edge.append((i1, i2))

# Lista com pontos que foram utilizados           
n_dots = list(set(np.array(lista_edge).flatten()))

# Lista com nome das colunas que foram utilizadas
n_name = [list(df_outlier.keys())[i] for i in n_dots]

## Plota correlação entre colunas

In [ ]:
plt.figure(figsize=(20,10))
G.add_edges_from(lista_edge)
pos=nx.spring_layout(G, k=0.40)
nx.draw(G, pos, with_labels=True, width=0.2, node_color='lightblue', alpha=0.6, node_size=600)
plt.title('Correlação entre colunas - ano 2020')

plt.legend()
plt.show()